In [1]:
import os
import dotenv

import pymongo
import pandas as pd

In [2]:
def connect_to_db():
    """Open the connection to the DB and return the collection
    Create collection with unique index, if there is not yet one"""
    # Load environment variables from .env file
    dotenv.load_dotenv()
    
    # Get MongoDB-URI
    mongodb_uri = os.getenv("MONGODB_URI")
    DBclient = pymongo.MongoClient(mongodb_uri)
    db = DBclient["MDM-Python-MeinProjekt"]

    return db["Energie"]

In [3]:
def extract_daily_energy():

    collection = connect_to_db()
    
    pipeline = [
        {
            '$addFields': {
                'date': {
                    '$substr': [
                        '$datetime', 0, 10
                    ]
                }
            }
        },
        {
            '$group': {
                '_id': '$date',
                'wind': {'$avg': '$Wind Onshore Generation'},
                'solar': {'$avg':'$Solar Generation'}, 
                'nuclear': {'$avg':'$Nuclear Generation'}, 
                'water_reservoir': {'$avg':'$Hydro Water Reservoir Generation'}, 
                'water_river': {'$avg':'$Hydro Run-of-river and poundage Generation'}, 
                'water_pump': {'$avg':'$Hydro Pumped Storage Generation'},  
            }
        }
    ]
    
    results = []
    for x in collection.aggregate(pipeline):
        results.append(x)
    
    df = pd.DataFrame(results)
    df = df.set_index(("_id"))
    df = df.set_index(pd.to_datetime(df.index).rename("date").tz_localize("UTC"))
    df = df.sort_index()
    df["total"] = df.sum(axis="columns")
    
    return df

In [4]:
def extract_hourly_energy():
    collection = connect_to_db()

    projection = {
        '_id': False,
        'datetime': "$datetime",
        'wind': '$Wind Onshore Generation',
        'solar': '$Solar Generation', 
        'nuclear': '$Nuclear Generation', 
        'water_reservoir': '$Hydro Water Reservoir Generation', 
        'water_river': '$Hydro Run-of-river and poundage Generation', 
        'water_pump': '$Hydro Pumped Storage Generation', 
    }

    results = collection.find(projection=projection)
    
    df = pd.DataFrame(results)
    df = df.set_index("datetime")
    df = df.set_index(pd.to_datetime(df.index))
    df = df.sort_index()
    df["total"] = df.sum(axis="columns")

    return df

In [5]:
df_daily = extract_daily_energy()
df_daily

,wind,solar,nuclear,water_reservoir,water_river,water_pump,total
date,,,,,,,
2015-02-08 00:00:00+00:00,1.666667,1.500000,3347.083333,NaN,NaN,NaN,3350.250000
2015-02-09 00:00:00+00:00,0.208333,2.500000,3345.625000,NaN,NaN,NaN,3348.333333
2015-02-10 00:00:00+00:00,0.708333,3.208333,3324.708333,NaN,NaN,NaN,3328.625000
2015-02-11 00:00:00+00:00,0.541667,4.583333,3337.875000,NaN,NaN,NaN,3343.000000
2015-02-12 00:00:00+00:00,0.625000,4.958333,3338.208333,NaN,NaN,NaN,3343.791667
...,...,...,...,...,...,...,...
2024-03-28 00:00:00+00:00,0.000000,0.000000,3014.208333,1265.083333,255.583333,781.916667,5316.791667
2024-03-29 00:00:00+00:00,0.000000,0.000000,3008.541667,994.875000,219.125000,848.500000,5071.041667
2024-03-30 00:00:00+00:00,0.000000,0.000000,3002.958333,992.041667,259.541667,714.666667,4969.208333


In [13]:
df_hourly = extract_hourly_energy()
df_hourly

,wind,solar,nuclear,water_reservoir,water_river,water_pump,total
datetime,,,,,,,
2021-10-14 00:00:00,15.0,0.0,1729.0,402.0,156.0,19.0,2321.0
2021-10-14 01:00:00,16.0,0.0,1729.0,453.0,166.0,31.0,2395.0
2021-10-14 02:00:00,15.0,0.0,1729.0,371.0,168.0,48.0,2331.0
2021-10-14 03:00:00,22.0,0.0,1729.0,389.0,164.0,82.0,2386.0
2021-10-14 04:00:00,21.0,0.0,1729.0,924.0,191.0,515.0,3380.0
...,...,...,...,...,...,...,...
2024-03-12 19:00:00,0.0,0.0,3005.0,2437.0,256.0,1872.0,7570.0
2024-03-12 20:00:00,0.0,0.0,3003.0,2228.0,182.0,1297.0,6710.0
2024-03-12 21:00:00,0.0,0.0,3001.0,1789.0,157.0,600.0,5547.0
